# Browsing Books - A Recomendation Engine for Books
For CMPT3520 Machine Learning II <br/>
Annabell Rodriguez, Laura Brin, Sandra Alex

## Introduction

### Business Problem

### Evaluation Metrics

## Broswing Books

In [ ]:
#Loading Libraries

In [ ]:
#Loading Dataset

## Content-Based Filtering

## Collaborative Filtering-Matrix Factorization

## Collaborative Filtering-autoencoder Deep Neural Network

## Recommendation System

## Performance Evaluation

## Conclusion